In [6]:
# YOLOLayer下有什么东西

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

In [7]:
mse_loss = nn.MSELoss(size_average=True) # Coordinate loss  loss(xi,yi)=(xi−yi)2
bce_loss = nn.BCELoss(size_average=True) # Confidence loss  (-(target[0]*math.log(lossinput[0])+(1-target[0])*math.log(1-lossinput[0])))
ce_loss = nn.CrossEntropyLoss() # Class loss 

/home/lsc/anaconda3/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [8]:
import numpy as np
x = np.ones((3, 416, 416))
x = torch.tensor(x)
FloatTensor = torch.cuda.FloatTensor if x.is_cuda else torch.FloatTensor
print(type(x), FloatTensor, x.is_cuda)

<class 'torch.Tensor'> <class 'torch.FloatTensor'> False


In [9]:
torch.cuda.is_available()

True

In [10]:
x = x.cuda()
FloatTensor = torch.cuda.FloatTensor if x.is_cuda else torch.FloatTensor
print(type(x), FloatTensor, x.is_cuda)

<class 'torch.Tensor'> <class 'torch.cuda.FloatTensor'> True


In [11]:
dir(x)

['__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__cuda_array_interface__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ilshift__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rfloordiv__',
 '__rmul__',
 '__rpow__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',


In [12]:
def parse_model_config(path):
    """Parses the yolo-v3 layer configuration file and returns module definitions"""
    file = open(path, 'r')
    lines = file.read().split('\n')
    lines = [x for x in lines if x and not x.startswith('#')]
    lines = [x.rstrip().lstrip() for x in lines] # get rid of fringe whitespaces
    module_defs = []
    for line in lines:
        if line.startswith('['): # This marks the start of a new block
            module_defs.append({})
            module_defs[-1]['type'] = line[1:-1].rstrip()
            if module_defs[-1]['type'] == 'convolutional':
                module_defs[-1]['batch_normalize'] = 0
        else:
            key, value = line.split("=")
            value = value.strip()
            module_defs[-1][key.rstrip()] = value.strip()

    return module_defs

In [54]:
modules_defs = parse_model_config("/home/lsc/a409/users/lisuicheng/Machine_learning/PyTorch-YOLOv3/config/yolov3.cfg")
modules_defs

[{'type': 'net',
  'batch': '16',
  'subdivisions': '1',
  'width': '416',
  'height': '416',
  'channels': '3',
  'momentum': '0.9',
  'decay': '0.0005',
  'angle': '0',
  'saturation': '1.5',
  'exposure': '1.5',
  'hue': '.1',
  'learning_rate': '0.001',
  'burn_in': '1000',
  'max_batches': '500200',
  'policy': 'steps',
  'steps': '400000,450000',
  'scales': '.1,.1'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '32',
  'size': '3',
  'stride': '1',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '64',
  'size': '3',
  'stride': '2',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '32',
  'size': '1',
  'stride': '1',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '64',
  'size': '3',
  'stride': '1',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'shortcut', 'from': '-3', 'activation

In [55]:
modules_defs = parse_model_config("/home/lsc/a409/users/lisuicheng/Machine_learning/PyTorch-YOLOv3/config/yolov3.cfg")
print(type(modules_defs), type(modules_defs[0]), type(modules_defs[1]), type(modules_defs[-1]))
hyperparams = modules_defs.pop(0)
output_filters = [int(hyperparams["channels"])]
hyperparams, output_filters

<class 'list'> <class 'dict'> <class 'dict'> <class 'dict'>


({'type': 'net',
  'batch': '16',
  'subdivisions': '1',
  'width': '416',
  'height': '416',
  'channels': '3',
  'momentum': '0.9',
  'decay': '0.0005',
  'angle': '0',
  'saturation': '1.5',
  'exposure': '1.5',
  'hue': '.1',
  'learning_rate': '0.001',
  'burn_in': '1000',
  'max_batches': '500200',
  'policy': 'steps',
  'steps': '400000,450000',
  'scales': '.1,.1'},
 [3])

In [28]:
modules_defs # module_defs是list, 包含的时字典， 字典中时网络的信息, pop(0)后，第一个字典没了

[{'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '32',
  'size': '3',
  'stride': '1',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '64',
  'size': '3',
  'stride': '2',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '32',
  'size': '1',
  'stride': '1',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '64',
  'size': '3',
  'stride': '1',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'shortcut', 'from': '-3', 'activation': 'linear'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '128',
  'size': '3',
  'stride': '2',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '64',
  'size': '1',
  'stride': '1',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '128',
 

# nn.ModuleList()
普通list中的子module并不能被主module所识别，而ModuleList中的子module能够被主module所识别。这意味着如果用list保存子module，将无法调整其参数，因其未加入到主module的参数中。

除ModuleList之外还有ParameterList，其是一个可以包含多个parameter的类list对象。在实际应用中，使用方式与ModuleList类似。

ModuleList可以包含很多个nn.Sequential()


In [30]:
module_list = nn.ModuleList()

In [52]:
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.list = [nn.Linear(3, 4), nn.ReLU()]
        self.module_list = nn.ModuleList([nn.Conv2d(3, 3, 3), nn.ReLU()])
    def forward(self):
        pass

model = MyModule()

print(model)

for name, param in model.named_parameters():
    print(name, param.size())

MyModule(
  (module_list): ModuleList(
    (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
  )
)
module_list.0.weight torch.Size([3, 3, 3, 3])
module_list.0.bias torch.Size([3])


# BN（batch normalization)

目的：Internal Covariate Shift问题（训练时，中间层的参数不是归一化的，网络的泛化能力减弱）

方法：一个batch中每一个feature map的同一个channal中的所有的元素求均值和方差，每个元素进行变换，然后乘以gamma加上beta,缩放参数γ和平移参数β是学习出来的（白化+偏移）

https://blog.csdn.net/u010402786/article/details/51233854

这篇文章讲的非常详细，gamma的定义为根号下标准差， beta时学习出来的

```https://blog.csdn.net/qq_14821323/article/details/80021150```

我们训练时使用一个minibatch的数据，因此可以计算均值和方差，但是预测时一次只有一个数据，所以均值方差都是0，那么BN层什么也不干，原封不动的输出。这肯定会用问题，因为模型训练时都是进过处理的，但是测试时又没有，那么结果肯定不对。

解决的方法是使用训练的所有数据，也就是所谓的population上的统计。原文中使用的就是这种方法，不过这需要训练完成之后在多出一个步骤。另外一种常见的办法就是基于momentum的指数衰减，这种方法就是我们下面作业要完成的算法。

公式如下：

running_mean = momentum * running_mean + (1 - momentum) * sample_mean

running_var = momentum * running_var + (1 - momentum) * sample_var

如果读者做过滤波，这和低通滤波器类似。每次更新时把之前的值衰减一点点（乘以一个momentum，一般很大，如0.9,0.99），然后把当前的值加一点点进去(1-momentum)。

当然极端的情况下这种方法计算出来的值和实际的平均值是有差异的，比如如下例子

100, 100, 100 , … 1, 1, 1

比如有两百个数据，100个100和100个1，那么平均值应该是55.5。如果采样不好的话，前面全是100，后面全是1，那么用这种算法计算的就接近1。反之如果前面全部是1，后面全是100，那么结果就接近100。当然一般情况我们的采样是均匀的，那么计算出来就是解决55.5。

In [59]:
# -*-coding:utf-8-*-
from torch import nn
import torch

m = nn.BatchNorm2d(3)  # bn设置的参数实际上是channel的参数
input = torch.randn(4, 3, 2, 2)
output = m(input)
# print(output)
a = (input[0, 0, :, :]+input[1, 0, :, :]+input[2, 0, :, :]+input[3, 0, :, :]).sum()/16
b = (input[0, 1, :, :]+input[1, 1, :, :]+input[2, 1, :, :]+input[3, 1, :, :]).sum()/16
c = (input[0, 2, :, :]+input[1, 2, :, :]+input[2, 2, :, :]+input[3, 2, :, :]).sum()/16
print(a.data, b.data, c.data)
print((m.running_mean.data[0],m.running_mean.data[1],m.running_mean.data[2]))
print(m)

tensor(-0.1891) tensor(0.1850) tensor(-0.1192)
(tensor(-0.0189), tensor(0.0185), tensor(-0.0119))
BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


In [60]:
# LeakyRelu:x < 0时的值为0.01或者自定义a

# nn.Upsample
上采样，不同于unpooling

还有一个反卷积
https://blog.csdn.net/u014722627/article/details/60574260

In [66]:
input = torch.arange(1, 5).view(1, 1, 2, 2).float()
print(input)
m = nn.Upsample(scale_factor=2, mode='nearest')
m(input)

tensor([[[[1., 2.],
          [3., 4.]]]])


tensor([[[[1., 1., 2., 2.],
          [1., 1., 2., 2.],
          [3., 3., 4., 4.],
          [3., 3., 4., 4.]]]])

In [72]:
# MaxUnpool
pool = nn.MaxPool2d(2, stride=2, return_indices=True)
unpool = nn.MaxUnpool2d(2, stride=2)
input = torch.tensor([[[[1., 2., 3., 4.],
                        [5., 6., 7., 8.],
                        [9., 10., 11., 12.],
                        [13., 14., 15., 16.]]]])
input

tensor([[[[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [75]:
output, indices = pool(input)
print(output)
print(indices) # 记录了pooling的位置
unpool(output, indices)

tensor([[[[ 6.,  8.],
          [14., 16.]]]])
tensor([[[[ 5,  7],
          [13, 15]]]])


tensor([[[[ 0.,  0.,  0.,  0.],
          [ 0.,  6.,  0.,  8.],
          [ 0.,  0.,  0.,  0.],
          [ 0., 14.,  0., 16.]]]])

In [76]:
# 反卷积
from torch import nn
from torch.nn import init
from torch.autograd import Variable

dconv = nn.ConvTranspose2d(in_channels=1, out_channels= 1,  kernel_size=2, stride=2, padding=1,output_padding=0, bias= False)
init.constant(dconv.weight, 1)
print(dconv.weight)

input = Variable(torch.ones(1, 1, 2, 2))
print(input)
print(dconv(input))

Parameter containing:
tensor([[[[1., 1.],
          [1., 1.]]]], requires_grad=True)
tensor([[[[1., 1.],
          [1., 1.]]]])
tensor([[[[1., 1.],
          [1., 1.]]]], grad_fn=<ThnnConvTranspose2DBackward>)


/home/lsc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  import sys
